In [1]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("./bases/complete_db.csv")

In [3]:
df = df[df["status"] == "Released"]

In [4]:
df.columns

Index(['Unnamed: 0', 'id', 'value_x', 'Title', 'Year', 'Rated', 'Released',
       'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language',
       'Country', 'Awards', 'Poster', 'Ratings', 'Metascore', 'imdbRating',
       'imdbVotes', 'imdbID', 'Type', 'DVD', 'BoxOffice', 'Production',
       'Website', 'Response', 'Season', 'Episode', 'seriesID', 'Error', 'id_x',
       'imdb_id', 'value_y', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id_y', 'origin_country',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'video', 'vote_average', 'vote_count',
       'belongs_to_collection.id', 'belongs_to_collection.name',
       'belongs_to_collection.poster_path',
       'belongs_to_collection.backdrop_path'],
      dtype='object')

In [5]:
#columns_to_ignore = [
#    "Poster", "Awards", "Season","video","DVD",
#    "homepage","poster_path","tagline","Website","Plot","Episode","Season","overview",   
#]

columns_to_use = {
    "Title", "Year", "Ratings", "runtime", "Genre", 
    "Ratings", "budget", 'revenue', 'popularity', "production_companies", "BoxOffice",
}

columns_to_drop = set(df.columns) - columns_to_use

df = df.drop(columns_to_drop, axis=1)

In [6]:
import ast

# Function to convert list of dicts into a dict
def extract_ratings(ratings_list):
    return {item['Source']: item['Value'] for item in ratings_list}

#df['Ratings'].apply(json.loads).apply(extract_ratings).apply(pd.Series)

df = pd.concat([
    df.drop(columns=["Ratings"]),
    df['Ratings'].apply(json.loads).apply(extract_ratings).apply(pd.Series)
], axis=1)

In [7]:
def extract_companies(companies_list):
    names = [item['name'] for item in companies_list]
    return ','.join(names)

df["production_companies"] = df["production_companies"].apply(json.loads).apply(extract_companies)

In [8]:
df = df[(df['budget'] > 0) & (df['revenue'] > 0)]
df['budget'] = df['budget']/1000_000
df['revenue'] = df['revenue']/1000_000


In [9]:
if 'Internet Movie Database' in df.columns:
    df['Internet Movie Database'] = df['Internet Movie Database'].str.replace('/10', '').astype(float) * 10

if 'Rotten Tomatoes' in df.columns:
    df['Rotten Tomatoes'] = df['Rotten Tomatoes'].str.replace('%', '').astype(float)

if 'Metacritic' in df.columns:
    df['Metacritic'] = df['Metacritic'].str.replace('/100', '').astype(float)


In [10]:
# First, clean the original Genre column
df['Genre'] = df['Genre'].str.strip()  # Remove leading/trailing spaces
df['Genre'] = df['Genre'].str.replace(' ,', ',')  # Remove spaces before commas
df['Genre'] = df['Genre'].str.replace(', ', ',')  # Remove spaces after commas

# Then split and normalize individual genres
df = df.drop('Genre', axis=1).join(
    df['Genre']
    .str.split(',')
    .explode()
    .str.strip()
    .rename('Genre')
)

# Optional: Check for unique genres to verify the cleanup
print("Unique genres after cleaning:")
print(sorted(df['Genre'].unique()))

Unique genres after cleaning:
['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western']


In [11]:
df.to_csv("bruno_db.csv", index=False)